In [1]:
pip install pennylane

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.1/56.1 kB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 28.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 930.0/930.0 kB 30.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 44.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 36.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 167.9/167.9 kB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.6/8.6 MB 21.8 MB/s eta 0:00:00


In [ ]:

import pennylane as qml
from pennylane import numpy as np
from pennylane.templates import RandomLayers
import tensorflow as tf
from tensorflow import keras
import matplotlib.pyplot as plt
from numba import njit
import numpy as np

# Hamiltonian matrix construction
@njit
def build_hamiltonian_matrix(num_qubits, matrix_size, J, K, h, g):
    matrix = np.zeros((matrix_size, matrix_size))
    for i in range(matrix_size):
        for j in range(matrix_size):
            H_value = 0
            for i1 in range(num_qubits):
                for j1 in range(num_qubits):
                    if j == i ^ (2 ** i1 + 2 ** j1):
                        H_value += K[i1, j1]
                    if j == i:
                        sign = 1
                        if (i & 2 ** i1) != 0:
                            sign = -sign
                        if (i & 2 ** j1) != 0:
                            sign = -sign
                        H_value += sign * J[i1, j1]

            for i1 in range(num_qubits):
                if j == i ^ (2 ** i1):
                    H_value += g[i1]

            if j == i:
                H_value += h
            matrix[i, j] = H_value
    return matrix

# Main part of the code
num_qubits = 6
matrix_size = 2**num_qubits
num_repetitions = 1000
Mz_values = np.zeros((num_repetitions, matrix_size))

eigenvalues = []
eigenvectors = []

for repetition in range(num_repetitions):
    np.random.seed(repetition)
    J = np.random.uniform(low=-1, high=0, size=(num_qubits, num_qubits))
    for i in range(num_qubits):
        for j in range(num_qubits):
            if i >= j:
                J[i, j] = 0
    J = (J + J.T)

    np.random.seed(repetition + 10)
    K = np.random.uniform(low=-1, high=1, size=(num_qubits, num_qubits))
    for i in range(num_qubits):
        for j in range(num_qubits):
            if i >= j:
                K[i, j] = 0
    K = (K + K.T)

    np.random.seed(repetition + 20)
    h = np.random.uniform(low=-0.04, high=0.04)

    np.random.seed(repetition + 30)
    g = np.random.uniform(low=-6, high=6, size=num_qubits)

    # Build the Hamiltonian matrix using the JIT-compiled function
    matrix = build_hamiltonian_matrix(num_qubits, matrix_size, J, K, h, g)

    eigenval, eigenvect = np.linalg.eigh(matrix)

    min_eigenval = np.min(eigenval)
    min_eigenvec = eigenvect[np.argmin(eigenval), :]

    eigenvalues.append(min_eigenval)
    eigenvectors.append(min_eigenvec)

eigenvectors_matrix = np.column_stack(eigenvectors)

Mzt = []
for column in eigenvectors_matrix.T:
    i = 0
    Mz = 0
    for component in column:
        for n_prime in range(num_qubits):
            if (2 ** n_prime) & i != 0:
                Mz += abs(component) ** 2 / num_qubits
        i += 1
    Mz = 2 * Mz - 1
    Mzt.append(Mz)

Mzt_row = np.array(Mzt)
final_data = np.vstack((eigenvectors_matrix, Mzt_row))

from sklearn.model_selection import train_test_split

# Convert data to TensorFlow tensors
features = np.array(eigenvectors)
labels = np.array(Mzt_row)

# Split the data into training, testing, and validation sets
features_train, features_temp, labels_train, labels_temp = train_test_split(features, labels, test_size=0.3, random_state=42)
features_val, features_test, labels_val, labels_test = train_test_split(features_temp, labels_temp, test_size=0.5, random_state=42)

# Print the shapes of the sets
print("Training set shapes:", features_train.shape, labels_train.shape)
print("Validation set shapes:", features_val.shape, labels_val.shape)
print("Testing set shapes:", features_test.shape, labels_test.shape)

#===========================================================================
#
#===========================================================================
# Assuming you have eigenvectors and Mzt_row available
eigenvectors = np.column_stack(eigenvectors)  # Transpose eigenvectors for correct shape
Mzt_row = np.array(Mzt_row)

import pennylane as qml
from pennylane import numpy as np
import tensorflow as tf
from tensorflow import keras
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split

# Quantum device setup
filter_size = 8 # The size of the filter for the quantum convolutional layer
stride = 2       # The stride for the convolution
dev = qml.device("default.qubit", wires=filter_size)

np.random.seed(42)
# Parameters for the parameterized gates
rand_params = np.random.uniform(high=2 * np.pi, size=(filter_size,))

# Define a custom quantum circuit function
@qml.qnode(dev)
def circuit(phi):
    # Apply parameterized Y rotations and CNOT gates
    for j in range(filter_size):
        qml.RY(phi[j], wires=j)  # Apply parameterized RY gate
    for j in range (filter_size):
        qml.RX(rand_params, wires=j )
    # Apply CNOT gates to create entanglement between qubits
    for j in range(filter_size - 1):
        qml.CNOT(wires=[j, j + 1])

    # Measure expectation values of Pauli-Z for each qubit in the filter
    return [qml.expval(qml.PauliZ(j)) for j in range(filter_size)]

# Quantum convolutional function with stride
def quanv(input_data, filter_size, stride):
    num_segments = (len(input_data) - filter_size) // stride + 1
    output = []

    for i in range(num_segments):
        start = i * stride
        segment = input_data[start:start + filter_size]

        if len(segment) == filter_size:
            out = circuit(segment)
            output.append(out)

    return np.array(output)

# Quantum preprocessing using the quanv function
q_train_features = np.array([quanv(x, filter_size, stride) for x in features_train])
q_val_features = np.array([quanv(x, filter_size, stride) for x in features_val])

# Reshape features for the Keras model
q_train_features = q_train_features.reshape(q_train_features.shape[0], -1)
q_val_features = q_val_features.reshape(q_val_features.shape[0], -1)

# Define a classical neural network model
def ClassicalModel():
    model = keras.models.Sequential([
        keras.layers.Flatten(),
        keras.layers.Dense(8, activation='relu'),  # Classical dense layer
        keras.layers.Dense(1)  # Regression output layer
    ])
    model.compile(optimizer='adam', loss='mse', metrics=['mae'])
    return model

# Define a quantum neural network model
def QuantumModel():
    model = keras.models.Sequential([
        keras.layers.Flatten(),
        keras.layers.Dense(1)  # Regression output layer
    ])
    model.compile(optimizer='adam', loss='mse', metrics=['mae'])
    return model

# Train the classical model
c_model = ClassicalModel()
c_history = c_model.fit(features_train, labels_train, validation_data=(features_val, labels_val), batch_size=32, epochs=30, verbose=2)

# Train the quantum model
q_model = QuantumModel()
q_history = q_model.fit(q_train_features, labels_train, validation_data=(q_val_features, labels_val), batch_size=32, epochs=30, verbose=2)

# Plot the results
fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(6, 9))

ax1.plot(q_history.history["val_mae"], "-ob", label="With quantum layer")
ax1.plot(c_history.history["val_mae"], "-og", label="Without quantum layer")
ax1.set_ylabel("Mean Absolute Error (MAE)")
ax1.set_xlabel("Epoch")
ax1.legend()

ax2.plot(q_history.history["val_loss"], "-ob", label="With quantum layer")
ax2.plot(c_history.history["val_loss"], "-og", label="Without quantum layer")
ax2.set_ylabel("Loss")
ax2.set_ylim(top=0.2)
ax2.set_xlabel("Epoch")
ax2.legend()

plt.tight_layout()
plt.show()
